In [15]:
import pandas as pd
import numpy as np
from binance.client import Client
from huobi.linear_swap.rest.account import Account
from huobi.linear_swap.rest.market import Market
from huobi.linear_swap.rest.order import Order
import ftx
from pybit import spot
import pybit

import os
import sys

In [16]:
Binance = {"API": "ojaWM7JF5nib3nY9hpiTIM9RZDTmVWIW5k5Z4HrY11CksrN18IJuJrP42WSed2X4",
           "Secret": "qkqY8QG9xiYQCvtctBBHhFqnUvz9mC1RiAxiSkU6zkTX3RisRwFc3KRINPWn91bz"}

Huobi = {"API": "cc5417d1-d6f5a369-cdgs9k03f3-1916a",
         "Secret": "c3680fc5-19f9569d-1e6fcce3-c60a0"}

FTX = {"API": "7DS8r5ipITuiR81i9Nov_Fi8jjxujDFaMWZvLW2_",
         "Secret": "gaHIllTYJEXqX8G3gUvgmIRE-WzL36rfHmBmpc-E"}
# Bybit = {"API": "NUxyoWG4BHAb7LTnxD", 
#          "Secret": "ndCqe0JoQxXIyyZ5ptSuQjlKOpmLDxlJMobo"}
# Bybit = {"API": "N2uSv0QzEOlWdfH9hA", 
#          "Secret": "4Js0R4PtfMHuX8MojpU0N0RL00yVoc4z7wlo"}
# Bybit = {"API": "2UoJAV4WyNuuWYJRXV", 
        #  "Secret": "Pe2GeAnydR3pGvjDCdxVbe1loZcrSwNjsuBP"}
# Bybit = {"API": "tT1bPqQ0gZCV7o8oZq", 
#          "Secret": "e7qDcXya61CuBcsbS5mOPy9310aeSO8f44xd"}
Bybit = {"API": "j5NyBnCaabZlWM5dV1", 
         "Secret": "NADuLGwN3DatdZNlolrCqWtWFlq1YEhAr4rE"}

In [109]:
Bybit_client = spot.HTTP(
    endpoint="https://api.bybit.com",
    api_key=Bybit["API"],
    api_secret=Bybit["Secret"]
)
FTX_client = ftx.FtxClient()


In [107]:
res = Bybit_client.place_active_order(
    symbol = "ETHUSDT",
    side = "Buy",
    type = "LIMIT_MAKER", 
    qty = 0.01,
    price = 1071,
    time_in_force="GoodTillCancel",
    # reduce_only=False,
    # close_on_trigger=False
    )

In [115]:
symbols = ["ETH/USDT", "ETH/BTC", "BTC/USDT"]
FTX_maker_charge = 0.001
unit = 100

BTC_Storage = 0
ETH_Storage = 0
USDT_Storage = 100
while True:
    FTX_orders = FTX_client.get_orderbook(symbols[0], 1)
    By_bid_0 = float(FTX_orders["bids"][0][0])
    By_ask_0 = float(FTX_orders["asks"][0][0])
    FTX_orders = FTX_client.get_orderbook(symbols[1], 1)
    By_bid_1 = float(FTX_orders["bids"][0][0])
    By_ask_1 = float(FTX_orders["asks"][0][0])
    FTX_orders = FTX_client.get_orderbook(symbols[2], 1)
    By_bid_2 = float(FTX_orders["bids"][0][0])
    By_ask_2 = float(FTX_orders["asks"][0][0])

    if (1 / By_ask_0 * By_bid_1 * By_bid_2 > 1 / (1 - FTX_maker_charge)**3):
        # Charging_Reward -= 3 * FTX_maker_charge * unit * By_ask_0
        # print("Buy ETH, Sell ETH to BTC, Sell BTC get USDT")
        USDT_Storage -= unit * By_ask_0
        ETH_Storage += unit

        BTC_Storage += ETH_Storage * By_bid_1
        ETH_Storage = 0

        USDT_Storage += BTC_Storage * By_bid_2
        BTC_Storage = 0
        # print("USDT:", USDT_Storage, "ETH:", ETH_Storage, "BTC", BTC_Storage, "Charging Reward:", Charging_Reward, "Profit:", USDT_Storage + Charging_Reward)
        print(By_bid_0, By_ask_0)

KeyboardInterrupt: 

In [116]:
Bybit_symbols = ["ETHUSDT", "ETHBTC", "BTCUSDT"]
Bybit_maker_charge = 0.0001
Charging_Reward = 0.0

FTX_maker_charge = 0.0002

unit = 0.05
# profit = 0


BTC_Storage = 0
ETH_Storage = 0
USDT_Storage = 100

while True:
    # print(Bybit_client.last_traded_price())
    # record =Bybit_client.public_trading_records(symbol=Bybit_symbols[0])["result"]
    # print(record[0]["price"], record[0]["qty"], record[0]["time"])

    By_orders_0 = Bybit_client.orderbook(symbol=Bybit_symbols[0])
    By_bid_0 = float(By_orders_0["result"]["bids"][0][0])
    By_ask_0 = float(By_orders_0["result"]["asks"][0][0])

    By_orders_1 = Bybit_client.orderbook(symbol=Bybit_symbols[1])
    By_bid_1 = float(By_orders_1["result"]["bids"][0][0])
    By_ask_1 = float(By_orders_1["result"]["asks"][0][0])

    By_orders_2 = Bybit_client.orderbook(symbol=Bybit_symbols[2])
    By_bid_2 = float(By_orders_2["result"]["bids"][0][0])
    By_ask_2 = float(By_orders_2["result"]["asks"][0][0])

    if (1 / By_ask_0 * By_bid_1 * By_bid_2 > 1 / (1 - Bybit_maker_charge)**3):
        Charging_Reward -= 3 * Bybit_maker_charge * unit * By_ask_0
        # print("Buy ETH, Sell ETH to BTC, Sell BTC get USDT")
        USDT_Storage -= unit * By_ask_0
        ETH_Storage += unit

        BTC_Storage += ETH_Storage * By_bid_1
        ETH_Storage = 0

        USDT_Storage += BTC_Storage * By_bid_2
        BTC_Storage = 0
        # print("USDT:", USDT_Storage, "ETH:", ETH_Storage, "BTC", BTC_Storage, "Charging Reward:", Charging_Reward, "Profit:", USDT_Storage + Charging_Reward)
        print(By_bid_0, By_ask_0)
    # break

    # if (1 / By_ask_2 / By_ask_1 * By_bid_0 > 1 / (1 - Bybit_maker_charge)**3):
    #     print("Buy BTC, Sell BTC to ETH, Sell ETH get USDT")
